In [1]:
import numpy as np
import cv2 as cv
from tqdm import tqdm

In [2]:
CALIBRATION_VIDEO_PATH = '../data/calibration.mp4'
CAM_MATRIX_PATH  = '../processed/K.npy'
DISTORTION_PATH  = '../processed/dist.npy'
CHECKERBOARD_WIDTH   = 9
CHECKERBOARD_HEIGHT  = 6
TERMINATION_CRITERIA = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [3]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

objp = np.zeros((CHECKERBOARD_WIDTH * CHECKERBOARD_HEIGHT, 3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD_WIDTH, 0:CHECKERBOARD_HEIGHT].T.reshape(-1,2)

In [4]:
camera_matrix = None
inspected = 0
detected  = 0

In [5]:
cap = cv.VideoCapture(CALIBRATION_VIDEO_PATH)
SKIP_FRAMES  = 4
skip_counter = SKIP_FRAMES

while cap.isOpened():
    ret, frame = cap.read()


    if not ret:
        break
    
    if skip_counter == 0:
        skip_counter = SKIP_FRAMES
        inspected += 1
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        if (camera_matrix == None):    
            camera_matrix = gray.shape[::-1]

        ret, corners = cv.findChessboardCorners(
            gray,
            (CHECKERBOARD_WIDTH, CHECKERBOARD_HEIGHT),
            flags=cv.CALIB_CB_FAST_CHECK
        )


        if (ret == True):
            detected+=1
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), TERMINATION_CRITERIA)
            imgpoints.append(corners2)
            #cv.drawChessboardCorners(frame, (CHECKERBOARD_WIDTH, CHECKERBOARD_HEIGHT), corners2, ret)
        else:
            pass
    else:
        skip_counter-=1

objpoints = np.array(objpoints)
imgpoints = np.array(imgpoints)
    
print('Recall', detected / inspected)
    
cap.release()
cv.destroyAllWindows()

Recall 0.9917184265010351
479 479


In [6]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, camera_matrix, None, None, flags = cv.CALIB_USE_LU)
np.save(CAM_MATRIX_PATH, mtx)
np.save(DISTORTION_PATH, dist)